<a href="https://colab.research.google.com/github/HoonC-corgi/Machine_Learning_SelfStudy/blob/main/05_2_Cross_Validation%2C_Grid_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 검증세트

In [19]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [20]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()   # 클래스 열을 타깃으로 사용, 나머지 열은 특성에 저장

In [21]:
# 훈련 세트와 테스트 세트를 나눔
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [22]:
# train_input, train_target을 다시 train_test_split() 메소드에 넣어 훈련 세트 sub_input, sub_target과 검증 세트 val_input, val_target을 만듦
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [23]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [24]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


# 교차 검증
      검증 세트를 떼어 내어 평가하는 과정을 반복하는 것
      훈련 세트를 세 파트로 나누어 교차 검증하는 3-폴드 교차 검증 등이 있음

In [33]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)   # cross_validaate() 메소드에 평가할 모델 객체를 첫 째 매개변수로 전달, 훈련 세트 전체를 전달하여 사용
print(scores)
# 순서대로 fit_time, score_time, test_score 키를 가진 딕셔너리를 반환함, 각 키는 5개의 숫자가 담기며 처음 2개의 키는 각각 모델을 훈련하는 시간과 검증하는 시간을 의미한다.
# 교차 검증으 ㅣ최종 점수는 test_score의 점수의 평균이다, 이는 test_socre지만 검증 폴드의 점수일 뿐이다

{'fit_time': array([0.01195145, 0.00797105, 0.00726223, 0.00748277, 0.00702977]), 'score_time': array([0.00145221, 0.0007441 , 0.00080967, 0.00076437, 0.00072193]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [34]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [35]:
# train_test_split() 메소드로 전체 데이터를 섞은 후 훈련 세틀르 준비했기 때문에 섞을 필요가 없지만, 교차 검증을 할 때 훈련 세트를 섞으려면 분할기를 지정해야 함.
# cross_validate()메소드는 기본적으로 회귀 모델일 경우 KFold 분할기를 사용, 분류 모델일 경우 StratifiedKFold를 사용한다.
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [37]:
# 훈련 세트를 섞은 후 10-폴드 교차 검증을 수행하면,
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


# 하이퍼파라미터 튜닝
      그리드 서치는 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행함 >> cross_validate() 호출이 필요없음

In [48]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}   # 매개변수의 최적값 찾기

In [49]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)   # cv 매개변수 디폴트는 5, >> 5-폴드 교차 검증을 수행, 즉 25개 모델 훈련

In [50]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [52]:
# 교차 검증에서 최적의 하이퍼파라미터를 찾으면 전체 훈련 세트로 모델을 다시 만들어야 하지만, 그리드 서치는 훈련이 끝나면 검증 점수가 가장 높은 모델의 매개변수 조합으로 자동으로 모델을 재훈련하고, 이를 gs 객체 best_estimator_ 속성에 저장함
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [53]:
# 최적의 매개변수는 best_params_ 속성에 저장
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [54]:
# 각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results_ 속성의 'mean_test_score'키에 저장됨.
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [57]:
# 넘파이 argmax() 메소드를 사용하면 가장 큰 값의 인덱스를 추출할 수 있음, 이를 통해 params 키에 저장된 매개변수를 출력하면,
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [64]:
# 보다 복잡한 조합을 탐색해보면,
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),    # 0.0001~0.001까지 0.0001을 더한 배열을 만들어 반복, 총 9개의 원소 생성
          'max_depth': range(5, 20, 1),   # 5~까지 1씩 증가, 15개의 값을 만듦
          'min_samples_split': range(2, 100, 10)   # 2~100까지 10씩 증가, 10개의 값
          }
# 즉 이 매개변수로 수행할 교차 검증 횟수는 9*15*10 =1350개임. 이에 기본 5-폴드 교차 검증을 수행하므로 6,750개의 교차 검증 수행

In [65]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [66]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [67]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


# 랜덤 서치
      매개변수 값의 목록 전달이 아닌 매개변수를 샘플링할 확률 분포 객체를 전달

In [68]:
from scipy.stats import uniform, randint

In [69]:
rgen = randint(0,10)

In [70]:
rgen.rvs(10)

array([2, 1, 9, 5, 7, 7, 6, 1, 4, 8])

In [72]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([113,  96,  89, 113,  88,  84, 104, 119,  89, 105]))

In [74]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.81912845, 0.02262344, 0.11357673, 0.98517082, 0.28967775,
       0.26976421, 0.87887403, 0.01703465, 0.18140045, 0.00850194])

In [87]:
# 탐색할 매개변수의 범위
params = {'min_impurity_decrease': uniform(0.0001, 0.001),    # 범위내 실숫값 샘플링
          'max_depth': randint(20, 50),                       # 범위내 정수값 샘플링
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [88]:
# 샘플링 회수는 사이킷런의 랜덤 서치 클래스 RandomizedSearchCV의 n_iter 매개변수에 지정
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f85dbf977f0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f85dbf97220>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f85dbf97d00>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f85dbfa53a0>},
                   random_state=42)

In [89]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [90]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [92]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86


# 전체 소스코드

# 교차 검증과 그리드 서치

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-2.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

## 검증 세트

In [ ]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [ ]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [ ]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [ ]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


## 교차 검증

In [ ]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01854587, 0.01207185, 0.01907134, 0.01821637, 0.01930857]), 'score_time': array([0.00080776, 0.00074434, 0.00079775, 0.00087142, 0.00078869]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼파라미터 튜닝

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [ ]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [ ]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [ ]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [ ]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [ ]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [ ]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [ ]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### 랜덤 서치

In [ ]:
from scipy.stats import uniform, randint

In [ ]:
rgen = randint(0, 10)
rgen.rvs(10)

array([2, 4, 6, 6, 7, 0, 0, 4, 7, 6])

In [ ]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([115,  94,  92,  98,  98,  94,  92,  98,  95, 124]))

In [ ]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([7.85256969e-01, 6.17903362e-01, 6.74630219e-01, 2.57740845e-01,
       1.55895938e-01, 7.99926438e-01, 3.65283219e-04, 9.16323805e-01,
       2.78694357e-01, 3.88077774e-01])

In [ ]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f934a490c10>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f934a5b03a0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f934a6085b0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f934a5b0040>},
                   random_state=42)

In [ ]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [ ]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.86


# 총 정리
  ## 키워드
  ### 검증세트
        하이퍼파라미터 튜닝을 위해 모델을 평가할 때, 테스트 세트를 사용하지 않기 위해 훈련 세트에서 다시 떼어 낸 데이터 세트
  
  ### 교차 검증
        훈련 세트를 여러 폴드로 나눈 다음 한 폴드가 검증 세트의 역할을 하고 나머지 폴드에서는 모델을 훈련함, 교차 검증의 점수는 이들의 평균

  ### 그리드 서치
        하이퍼파라미터 탐색을 자동화해 주는 도구, 탐색할 매개변수를 나열하면 교차 검증을 수행하여 가장 좋은 검증 점수의 매개변수 조합을 선택하고 이를 최종 모델로 훈련함

  ### 랜덤 서치
        연속된 매개변수 값을 탐색할 때 유용함, 탐색할값을 직접 나열하지 않고 탐색 값을 샘플링할 수 있는 확률 분포 객체를 전달함, 지정된 횟수만큼 샘플링하여 교차 검증을 수행하기에 시스템 자원이 허락하는 만큼 탐색량을 조절할 수 있음



  ## 핵심 패키지와 함수
  
  ### 사이킷런
  #### corss_validate()
        교차 검증을 수행하는 메소드
        첫 번째 매개변수에 교차 검증을 수행할 모델 객체를 전달, 두 세 번째 매개변수에 특성과 데이터를 전달
        회귀일 경우 KFold 클래스, 분류일 경우 StratifiedKFold 클래스를 사용하여 5-폴드 교차 검증을 수행함
        n_jobs 매개변수는 교차 검증 수행시 사용할 CPU 코어수를 지정, 디폴트는 -1로 모두 사용
        return_train_score 매개변수를 True 로 지정하면 훈련 세트의 점수도 반환, 디폴트는 False

  #### GridSearchCV
        교차 검증으로 하이퍼파라미터 탐색을 수행, 최상의 모델을 찾은 후 훈련 세트 전체를 사용해 최종 모델을 훈련함
        첫 번재 매개변수로 그리드 서치 수행 모델 객체, 두 번째 ㅁ개ㅐ변수에 탐색할 모델의 매개변수와 값을 전달

  #### RandomizedSearchCV
        교차 검증으로 랜덤한 하이퍼파라미터 탐색을 수행함, 최상의 모델을 찾은 후 훈련 세트 전체를 사용해 최종 모델을 훈련함
        첫 번째 매개변수로 그리드 서치를 수행 모델 객체를 전달, 두 번째 매개변수에는 탐색할 모델의매개변수와 확률 분포 객체를 전달


  
  ## 팁
  ### 검증 세트의 비율
        훈련 세트: 검증 세트: 테스트 세트 = 6 : 2 : 2

  ### cross_validate()
        훈련 세트를 섞어 폴드를 나누지 않음.
        train_test_split()을 이용한 경우 따로 섞지 않아도 되지만, 교차 검증시 섞기 위해서는 splitter를 지정해주어야 함. 회귀 모델일 경우 KFold, 분류 모델일 경우 StratifiedKFold를 사용

  ### 최적의 매개변수
        max_depth의 최적값은 min_samples_split 매개변수의 값이 바뀌면 함께 달라지기 때문에 동시에 바꿔가며 최적의 값을 찾아야 함.

  ### 그리드 서치
        1. 먼저 탐색할 매개변수를 지정
        2. 그다음 훈련 세트에서 그리드 서치를 수행하여 최상의평균 검증 점수가 나오는 매개변수 조합을 찾음, 이는 그리드 서치 객체에 저장됨
        3. 그리드 서치는 최상의 배개변수에서 교차 검증에 사용한 훈련 세트가 아닌, 전체 훈련 세트를 사용해 최종 모델을 훈련하고, 이 모델 역시 그리드 서치 객체에 저장함

  ### 교차 검증의 수행 횟수
        각 매개변수 범위에 따른 반복 횟수들의 곱
        
        ex) # 탐색할 매개변수의 범위
        params = {'min_impurity_decrease': uniform(0.0001, 0.001),
                  'max_depth': randint(20, 50),
                  'min_samples_split': randint(2, 25),
                  'min_samples_leaf': randint(1, 25),}
          
          
          9*15*10 = 1350 번 반복

## 확인문제

In [ ]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f934a490c10>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f934a5b03a0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f934a6085b0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f934a5b0040>},
                   random_state=42)

In [ ]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077
